In [ ]:
#Мироненко Александр
#Позже сделаю версию .py

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [2]:
class Interpolator:
    """Linear interpolator.
    """

    @staticmethod
    def interpolate(x_list: list, y_list: list, z: float):
        """Linear interpolate.
        Parameters
        __________
        x_list : list
            x values.
        y_list: list
            y values.
        z: float
            Interpolate in that point z.
        Returns
        _______
        float
            Interpolate value.
        Raises
        ______
        ValueError
            x_list must be sorted ASC.
        """
        if x_list != sorted(x_list):
            raise ValueError('x_list must be sorted ASC')
        for index, element in enumerate(x_list):
            if z <= element:
                delta = (z - x_list[index - 1]) / (x_list[index] - x_list[index - 1])
                answer = y_list[index - 1] + (y_list[index] - y_list[index - 1]) * delta
                break
        return answer

In [3]:
class curve_discount_swap_points:
    """класс для построения кривой дисконтирования для рубля и кривой своп пойнтов для доллар рубля.
    """
    def __init__(self,current_data):
        self.sw_points_time,self.sw_points_value = None,None
        self.current_data = current_data
#         пока ничего не делаем с USD_RATE
        
    @staticmethod
    def discount_factor(data):
        return [[i,0.99] for i in data]
    
    @staticmethod
    def preparation_SW_data(SW_data):
        term = {"ON":pd.Timedelta("0 D").days,"W":pd.Timedelta("1 W").days,"M":pd.Timedelta("30 D").days,"Y":pd.Timedelta("1 Y").days}
        
        time = [term["ON"] if i=="ON" else int(i[:-1])*term[i[-1]]  for i in SW_data["Term"]]
        
        points = SW_data["SW POINTS"]
        return time,points

    def sw_points_inter(self,data):
        data_new = list(map(pd.to_datetime,data))
        data_new = [(i.date()-self.current_data).days for i in data_new]
        return [[data[i],Interpolator.interpolate(self.sw_points_time,self.sw_points_value,data_new[i])] for i in range(len(data_new))]

In [4]:
data = pd.read_csv('RUB swap points.csv', delimiter=',').loc[:,["Term","SW POINTS"]]

In [5]:
My_class = curve_discount_swap_points(pd.to_datetime('today').date())
My_class.sw_points_time,My_class.sw_points_value = My_class.preparation_SW_data(data)

In [6]:
date = ["11.11.21","01.04.22","01.05.24"]
My_class.sw_points_inter(date)

[['11.11.21', 589.2857142857142],
 ['01.04.22', 7906.133333333333],
 ['01.05.24', 108388.8602739726]]

In [8]:
My_class.discount_factor(date)

[['11.11.21', 0.99], ['01.04.22', 0.99], ['01.05.24', 0.99]]

In [7]:
data

,Term,SW POINTS
0,ON,131
1,1W,933
2,2W,2269
3,1M,4156
4,2M,8174
5,3M,12766
6,6M,25723
7,9M,39130
8,12M,52760
9,2Y,101653
